In [1]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 678.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached python_slugify-8.0.1-py2.py3-none-any.whl (9.7 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Created wheel for kaggle: filename=kaggle-1.6.3-py3-none-any.whl size=111919 sha256=8eb2d50cfb1f103b019ac1f6e637f34e1995fee84f37e2ae0d297f3b019b60c6
  Stored in directory: /Users/rastkopavlovic/Library/Caches/pip/wheels/c5/94/5b/08d5bb9b7b78401fa26da264ef32d72bfbd9cb74641c65169b
Successfully built kaggle


In [ ]:
# For Google Colab only
from google.colab import files
files.upload()  # Upload kaggle.json here

In [ ]:
# kaggle key activation

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Data retrieval
!kaggle datasets download -d trolukovich/food11-image-dataset
!unzip -q food11-image-dataset.zip -d dataset

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

In [ ]:
def show_images(images, titles=None, ncols=3):
    n = len(images)
    nrows = n // ncols + (1 if n % ncols else 0)
    fig, axs = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows))
    axs = axs.flatten()
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        axs[i].imshow(img)
        axs[i].axis('off')
        if titles:
            axs[i].set_title(titles[i])
    plt.tight_layout()
    plt.show()

# Show first few images from a specific food category
food_category = 'dataset/training/0'  # Change as needed
image_files = [os.path.join(food_category, f) for f in os.listdir(food_category)[:9]]
show_images(image_files)

In [ ]:
# Analyzing dataset distribution.

categories = os.listdir('dataset/training')
counts = [len(os.listdir(os.path.join('dataset/training', c))) for c in categories]

sns.barplot(x=categories, y=counts)
plt.xlabel('Food Categories')
plt.ylabel('Number of Images')
plt.title('Distribution of Food Categories in Training Set')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

def plot_image_size_distribution(directory):
    widths, heights = [], []
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)
        if os.path.isdir(subdir_path):
            for file in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, file)
                with Image.open(file_path) as img:
                    width, height = img.size
                    widths.append(width)
                    heights.append(height)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(widths, bins=50, color='skyblue')
    plt.title('Width Distribution')
    plt.xlabel('Width')
    plt.ylabel('Number of Images')

    plt.subplot(1, 2, 2)
    plt.hist(heights, bins=50, color='salmon')
    plt.title('Height Distribution')
    plt.xlabel('Height')
    plt.show()

# Example usage
plot_image_size_distribution('dataset/training')

In [ ]:
import random

def show_sample_images_from_each_category(directory, samples_per_category=3):
    categories = os.listdir(directory)
    fig, axs = plt.subplots(len(categories), samples_per_category, figsize=(15, len(categories)*3))

    for i, category in enumerate(categories):
        category_path = os.path.join(directory, category)
        image_files = random.sample(os.listdir(category_path), samples_per_category)
        image_files = [os.path.join(category_path, f) for f in image_files]

        for j, img_path in enumerate(image_files):
            img = Image.open(img_path)
            axs[i, j].imshow(img)
            axs[i, j].axis('off')
            if j == 0:
                axs[i, j].set_ylabel(category)

    plt.tight_layout()
    plt.show()

# Example usage
show_sample_images_from_each_category('dataset/training')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Set your dataset directories
train_dir = 'dataset/training'
validation_dir = 'dataset/validation'
test_dir = 'dataset/evaluation'

# Define image size and batch size
img_height, img_width = 224, 224  # You can adjust these values as per your model's requirement
batch_size = 32

# Rescaling factor for normalization (1./255 is common for RGB images)
rescale_factor = 1./255

# Create Image Data Generators for resizing and normalization
train_datagen = ImageDataGenerator(
    rescale=rescale_factor,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=rescale_factor)

# Load images from directory and apply transformations
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' for binary classification
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' for binary classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' for binary classification
)
